In [5]:
from langchain_community.document_loaders.generic import GenericLoader
from langchain_community.document_loaders.parsers import LanguageParser
from langchain_text_splitters import Language, RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings, ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_classic.chains import create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from git import Repo

In [ ]:
repo_path = "./repo"

In [7]:
repo = Repo.clone_from("https://github.com/langchain-ai/langchain", to_path=repo_path)

In [10]:
loader = GenericLoader.from_filesystem(
  repo_path + "/libs/core/langchain_core",
  glob="**/*",
  suffixes=[".py"],
  exclude=["**/non-utf-8-encoding.py"],
  parser=LanguageParser(language=Language.PYTHON, parser_threshold=500)
)

documents = loader.load()
len(documents)

584

In [11]:
python_splitter = RecursiveCharacterTextSplitter.from_language(
  language=Language.PYTHON,
  chunk_size=2000,
  chunk_overlap=200,
)

texts = python_splitter.split_documents(documents)
len(texts)

1636

In [14]:
db = Chroma.from_documents(
  texts,
  embedding=OllamaEmbeddings(model="nomic-embed-text:v1.5"),
)

retriever = db.as_retriever(
  search_type="mmr", 
  search_kwargs={"k": 8}
)

In [16]:
llm = ChatOllama(
  model='phi3:mini',
  max_tokens=200,
)

In [18]:
prompt = ChatPromptTemplate.from_messages(
  [
    (
      "system",
      "You are an experienced code reviewer. Provide detailed information about the code review and suggestions for improvement based on the context provided below:\n\n{context}"
    ),
    (
      "user", 
      "{input}"
    ),
  ]
)

document_chain = create_stuff_documents_chain(llm=llm, prompt=prompt)

retrieveval_chain = create_retrieval_chain(retriever, document_chain)

In [23]:
response = retrieveval_chain.invoke({"input": "You can review and suggest improvements to the RunnableBinding code."})

In [24]:
print(response['answer'])

Certainly! However, please note that without specific details on what "RunnableBinding" entails or its context within your project, I'll provide a general set of suggestions for improving runnable binding in Python applications using dataclasses and type annotations:

```python
from abc import ABC, abstractmethod
from typing import Any, NamedTuple, Optional
import json

# Define custom exceptions if needed.
class InvalidParameterError(Exception): pass

@dataclass
class RunnableBinding:
    """Represents the binding for a runnable component."""
    
    tool_name: str  # Name of the Tool to execute.
    input_parameters: dict[str, Any] = field(default_factory=dict)
    log_message: Optional[str] = None
    
    def __post_init__(self):
        self._validate_input()
        
    def _validate_input(self):
        if not isinstance(self.tool_name, str) or len(self.tool_name.strip()) == 0:
            raise InvalidParameterError("Tool name must be a non-empty string.")
    
    @classmeth